# Basic database in MongoDB using PyMongo
**Author**: Edwin Barahona | July 2021

---

## Introduction

In this notebook I am presenting a series of exercises for MongoDB using Python.

**Level**: Intermediate

**Topics**:
* Data import
* Basic queries
* Aggregate functions
    * Count
    * gt, gte
    * lt, lte
    * sort
* Pipelines
    * match, group, redact

**Prerequisites**:
* **MongoDB 4.4** - *Locally installed or mongodb atlas*
* **PyMongo** and **bson** libraries

**Assumptions**:
* You have basic knowledge of both Python and Mongodb

**References**:

MongoDB documentation : https://docs.mongodb.com/manual/tutorial/getting-started/
PyMongo documentation : https://pymongo.readthedocs.io/en/stable/api/index.html
MongoDB Atlas: https://www.mongodb.com/atlas/database

## Part 1: Import data

In this part we will initialize the mongodb client, create a database and collection and
import data from two JSON files

In [ ]:
# Import bson and pymongo

from bson import json_util
from pymongo import MongoClient

In [ ]:
# set the json documents path

books_path = 'resources/act-2-books.json'
companies_path = 'resources/act-2-companies.json'

In [ ]:
# Now we load the json content using json_util.loads

books = []
with open(books_path, 'r') as jlist:
    for line in jlist:
        books.append(json_util.loads(line))

companies = []
with open(companies_path, 'r', encoding='utf-8') as jlist:
    for line in jlist:
        companies.append(json_util.loads(line))

In [ ]:
# Start mongodb client
client = MongoClient('localhost', 27017) # for this example mongo was installed locally

# In this step we define a db called 'misc' and two collections 'books' and 'companies'
db = client.misc
books_col = db.books
companies_col = db.companies

# Load the data to mongo using insert_many
books_col.insert_many(books)
companies_col.insert_many(companies)

## Part 2: Basic read operations

In this part we are performing basic read operations to our recently created database

In [ ]:
# import pprint to call the 'pretty print' this is used for display purposes

import pprint

# Print a sample element from the 'books' collection
pprint.pprint(books_col.find_one())

# Print a sample element from the 'companies' collection
pprint.pprint(companies_col.find_one())

In [ ]:
# Print the unique categories in the 'books' collection
print('Unique categories: ')
pprint.pprint(books_col.distinct('categories'))

# Print the unique statuses in the 'books collection
print('\nUnique statuses: ')
pprint.pprint(books_col.distinct('status'))

In [ ]:
# show the records with longDescription starting with the letter A
# display only title and longDescription fields
for record in books_col.find(
        {"longDescription": {"$gte": "A", "$lt": "B"}},
        {"title": 1, "longDescription": 1}):
    pprint.pprint(record)

# Find the books that have exactly 2 authors and are published, show only the fields
# title, longDescription, status and authors
for record in books_col.find({"authors": {"$size": 2}, "status": "PUBLISH"},
        {"_id": 0, "title": 1, "longDescription": 1, "status":1, "authors":1}):
    pprint.pprint(record)

## Part 3: Advanced reading operations

### Collection Stats

In [ ]:
# Print the size and count of elements in the collection books

stats = db.command('collstats','books')
pprint.pprint(stats['size'])
pprint.pprint(stats['count'])

# Print the size and count of elements in the collection companies

stats = db.command('collstats','companies')
pprint.pprint(stats['size'])
pprint.pprint(stats['count'])

### Aggregate functions
#### *Count, gt, gte, lte*

In [ ]:
# books with more than 200 pages
pprint.pprint(books_col.count_documents({"pageCount": {"$gt": 200}}))
# books with page count between 300 and 600
pprint.pprint(books_col.count_documents({"pageCount": {"$gte": 300, "$lte":600}}))
# books with 0 pages
pprint.pprint(books_col.count_documents({"pageCount": 0}))
# books with at least 1 page
pprint.pprint(books_col.count_documents({"pageCount": {"$gt": 0}}))
# books with status publish
pprint.pprint(books_col.count_documents({"status": "PUBLISH"}))
# books with status meap
pprint.pprint(books_col.count_documents({"status": "MEAP"}))

#### *Or* function, *And* function

In [ ]:
# find companies with  category code web o mobile
for document in companies_col.find(
        {"$or": [{"category_code": "web"}, {"category_code": "mobile"}]},
        {"name": 1, "category_code": 1, "number_of_employees": 1, "founded_year": 1}):
    pprint.pprint(document)

In [ ]:
# list the url, twitter username and number of employees  of companies that dedicate to
# videogames or music that were funded between 2001 and 2005
for document in companies_col.find(
        {"$and": [
            {"$or": [{"category_code": "games_video"},
                     {"category_code": "music"}]},
            {"founded_year": {"$gte": 2000, "$lte": 2005}},
            {"number_of_employees": {"$gt": 500}}]},
        {"_id": 0, "name": 1, "category_code": 1, "number_of_employees": 1, "founded_year": 1, "homepage_url": 1, "twitter_username": 1}):
    pprint.pprint(document)

In [ ]:
# list the emails of the videogames companies that were acquired in 2007 with a price over 20 million USD
# that have offices in Culver City
for document in companies_col.find(
        {"$and": [
            {'category_code': 'games_video'},
            {'acquisition.acquired_year': 2007},
            {'acquisition.price_amount': {'$gte': 20000000}},
            {'offices.city': 'Culver City'}]},
        {"_id": 0, "name": 1, 'email_address': 1}):
    pprint.pprint(document)

#### *Sort*

In [ ]:
# find the companies that dedicate to videogames sort in descending order by the founding year
for document in companies_col.find(
        {"category_code": "games_video"},
        {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1}).sort("founded_year", -1):
    pprint.pprint(document)


### Pipelines

In [ ]:
# List the companies with exactly 2 offices and both located in San Francisco
pipeline = [{"$match": {"offices": {"$elemMatch": {"city": 'San Francisco'}}}},
            {"$group": {"_id": "$name", "san_francisco_offices": {"$sum": 1}}},
            {"$redact":  {"$cond": [{"$gte": ['$san_francisco_offices', 2]}, '$$KEEP', '$$PRUNE']}}]
for record in companies_col.aggregate(pipeline):
    pprint.pprint(record)
